# Image Classification with ResNet50
A residual neural network (https://arxiv.org/abs/1512.03385) is an artificial neural network (ANN) of a kind that builds on constructs known from pyramidal cells in the cerebral cortex. Residual neural networks do this by utilizing skip connections, or shortcuts to jump over some layers. 

In this playground we load a pre-trained ResNet50 model and do some inference and evaluation on ImageNet dataset.

Note: Fireball uses the OpenCV python package to process images in the ImageNet dataset. If this package is not installed already, you can just run the following command in a new cell and restart the kernel.

```
%pip install opencv-python
```

## Create an ImageNet dataset
Lets first load the ImageNet dataset and see dataset statistics.

**Note 1:** ResNet50 uses the "Crop256Cafe" pre-processing for the images. This pre-processing first resizes the image (keeping the aspect ratio) so that its smaller dimension is 256. It then crops a 224x224 image from the center of the resized image. The image is in BGR format and the values are normalized using mean values 103.939, 116.779, 123.68 for blue, green, and red respectively. Please refer to Fireball's ```imagenet.py``` file for more information about different types of pre-processing supported by Fireball.

**Note 2:** If the ImageNet dataset is not available on this machine, the following code can take a long time the first time it is executed as the dataset needs to be downloaded and intialized. Please be patient and avoid interrupting the process during the download.


In [1]:
import numpy as np
import time, os
from fireball import Model, Block, myPrint
from fireball.datasets.imagenet import ImageNetDSet

gpus="0,1,2,3"

# Preparing the dataset and model (Downloading them if necessary)
ImageNetDSet.download()
Model.downloadFromZoo("ResNet50.fbm", "./Models/")

myPrint('\nPreparing ImageNet dataset ... ', False)
trainDs,testDs = ImageNetDSet.makeDatasets('Train,Test', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)
myPrint('Done.')
ImageNetDSet.printDsInfo(trainDs, testDs)

Creating folder "./Models/" ...
  Success!

Preparing ImageNet dataset ... Done.
ImageNetDSet Dataset Info:
    Number of Classes .............................. 1000
    Dataset Location ............................... /home/shahab/data/ImageNet/
    Number of Training Samples ..................... 1281167
    Number of Test Samples ......................... 50000
    Sample Shape ................................... (224, 224, 3)


## Create a ResNet50 Fireball model and print the model information
Let's load the model information from a pre-trained fireball model and print information about different layers of the model. For your information, the ResNet50's layer info text and blocks are as follows. Since we already have a trained fbm file for ResNet50, we don't need them here.

```
blocks = [ 
    Block('RES1|k_kernel_ixi,o_outSizes_i*3,s_stride_ixi_1|' +
          'add|' +
          'CONV_K1_S%s_O%o0_Pv,BN:ReLU,CONV_K%k_S1_O%o1_Ps,BN:ReLU,CONV_K1_S1_O%o2,BN;ID'),
    Block('RES2|k_kernel_ixi,o_outSizes_i*3,s_stride_ixi_1|' +
          'add|' +
          'CONV_K1_S%s_O%o0_Pv,BN:ReLU,CONV_K%k_S1_O%o1_Ps,BN:ReLU,CONV_K1_S1_O%o2,BN;'+
          'CONV_K1_S%s_O%o2_Pv,BN') ]

layers = ('IMG_S224;CONV_K7_O64_S2_P3,BN:ReLu:MP_K3_S2_P1;'  +              # Stage 1
          'RES2_K3_O64/64/256_S1:ReLU,2*RES1_K3_O64/64/256:ReLU;' +         # Stage 2
          'RES2_K3_O128/128/512_S2:ReLU,3*RES1_K3_O128/128/512:ReLU;' +     # Stage 3
          'RES2_K3_O256/256/1024_S2:ReLU,5*RES1_K3_O256/256/1024:ReLU;' +   # Stage 4
          'RES2_K3_O512/512/2048_S2:ReLU:DO,' +
          'RES1_K3_O512/512/2048:ReLU:DO,RES1_K3_O512/512/2048:ReLU:GAP;' + # Stage 5
          'FC_O1000,CLASS_C1000')                                           # Stage 6
```


In [2]:
model = Model.makeFromFile("Models/ResNet50.fbm", testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()


Reading from "Models/ResNet50.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      9,472      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      76,928     
S2_L2_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      71,552     
S2_L3_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      71,552     
S3_L1_RES2       56 56 256     2 Paths, 

## A quick inference demo
Now let's show how this model can be used to classify an image. Here we are using a JPEG image of a coffee mug.
The function ```getPreprocessedImage```, loads, scales, and preprocesses the image before returning it as numpy array of floating point numbers. We can pass the preprocessed image to the ```inferOne``` function to get the probabilities for each one of 1000 classes. We then print the top-3 classes with highest probabilities.

In [3]:
imageFileName = 'CoffeeMug.jpg'
image = testDs.getPreprocessedImage(imageFileName)
classProbs = model.inferOne(image, returnProbs=True)
top3Indexes = np.argsort(classProbs)[-3:][::-1]   # Indexes of classes with 3 highest probs (decreasing order)
top3Porbs = classProbs[top3Indexes]
print('Top-3 Classes (For "%s"):'%(imageFileName))
for i in range(3):
    print('    %s (%f)'%(ImageNetDSet.classNames[top3Indexes[i]], top3Porbs[i]))

Top-3 Classes (For "CoffeeMug.jpg"):
    coffee_mug (0.497880)
    cup (0.264278)
    pitcher (0.110466)


## Evaluating the model
This code runs inference on all images in the ImageNet dataset and compares the results with the ground truth labels in the testDs. The accuracy of the model is then printed.

In [4]:
myPrint('Running inference on %d Test Samples (batchSize:%d, %d towers) ... '%(testDs.numSamples,
                                                                               testDs.batchSize,
                                                                               len(model.towers)))
results = model.evaluate(topK=5)    # Calculate and print top-5 accuracy as well as the default top-1.

Running inference on 50000 Test Samples (batchSize:256, 4 towers) ... 
  Processed 50000 Sample. (Time: 59.68 Sec.)                              

Observed Accuracy: 0.747360
Top-5 Accuracy:   0.919360


## Where do I go from here?

[Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)

[Pruning ResNet50 Model](ResNet50-Prune.ipynb)

[Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)

[Exporting ResNet50 Model to ONNX](ResNet50-ONNX.ipynb)

[Exporting ResNet50 Model to CoreML](ResNet50-CoreML.ipynb)

[Exporting ResNet50 Model to TensorFlow](ResNet50-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)
